In [1]:
# try EDA software-> sweetviz, dataprep(terrible), dtale, profiling, lux

In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#from dataprep.eda import plot, plot_correlation, plot_missing (terrible)
#import plotly.express as px
#import sweetviz as sv
import dtale as dt
#import lux as lx
import pandas_profiling as pp


### Load the data

In [3]:
#load data
df = pd.read_csv('/Users/luis/Desktop/Py/mlbookcamp-code/course-zoomcamp/02-regression/homework_data/AB_NYC_2019.csv')

### Trying some EDA libraries

In [4]:
#use dataprep

#plot_missing(df)
#plot_correlation(df)
#plot(df)


In [5]:
#use sweetviz

#advert_report = sv.analyze(df)
#advert_report.show_html('Advertising.html')

In [6]:
#use dtale

#dt.show(df, ignore_duplicate=True)

In [7]:
#use lux
#df

In [8]:
#use profiling
#pp.ProfileReport(df_clean, title='DataFrame Profiling Report', html={'style':{'full_width':True}})

### Prepare the data

In [9]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [10]:
string = list(df.dtypes[df.dtypes=='object'].index)
string

['name',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'room_type',
 'last_review']

In [11]:
# go thru every string and replace empty spaces
for col in string:
    df[col] = df[col].str.lower().str.replace(' ', '_')

### Checking Price variable

In [12]:
# performd EDA on price variable
sns.histplot(df.price, bins=50)



<AxesSubplot:xlabel='price', ylabel='Count'>

In [13]:
sns.histplot(np.log1p(df.price.values))

<AxesSubplot:xlabel='price', ylabel='Count'>

### Choosing just the columns i need

* `'latitude'`,
* `'longitude'`,
* `'price'`,
* `'minimum_nights'`,
* `'number_of_reviews'`,
* `'reviews_per_month'`,
* `'calculated_host_listings_count'`,
* `'availability_365'`

In [14]:
df_clean = df.copy()

In [15]:
df_clean = df[['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']]
df_clean

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64749,-73.97237,149,1,9,0.21,6,365
1,40.75362,-73.98377,225,1,45,0.38,2,355
2,40.80902,-73.94190,150,3,0,NaN,1,365
3,40.68514,-73.95976,89,1,270,4.64,1,194
4,40.79851,-73.94399,80,10,9,0.10,1,0
...,...,...,...,...,...,...,...,...
48890,40.67853,-73.94995,70,2,0,NaN,2,9
48891,40.70184,-73.93317,40,4,0,NaN,2,36
48892,40.81475,-73.94867,115,10,0,NaN,1,27
48893,40.75751,-73.99112,55,1,0,NaN,6,2


### Question 1

Find a feature with missing values. How many missing values does it have?

Sol: 10052

In [16]:
#find how many missing values
df_clean.isnull().sum()

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

### Question 2

What's the median (50% percentile) for variable 'minimum_nights'?

Sol: 3

In [17]:
df_clean.minimum_nights.describe()

count    48895.000000
mean         7.029962
std         20.510550
min          1.000000
25%          1.000000
50%          3.000000
75%          5.000000
max       1250.000000
Name: minimum_nights, dtype: float64

### Split the data

* Shuffle the initial dataset, use seed `42`.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Make sure that the target value ('price') is not in your dataframe.
* Apply the log transformation to the price variable using the `np.log1p()` function.

In [18]:
n = len(df_clean)
#---
n_val = int(0.2*n)
n_test = int(0.2*n)
n_train = int(n - (n_val+n_test))
#---
n_total = int(n_val+n_train+n_test)


In [19]:
n, n_val, n_test, n_train, n_total

(48895, 9779, 9779, 29337, 48895)

In [20]:
idx = np.arange(n)
#---
np.random.seed(42)
np.random.shuffle(idx)

In [21]:
df_train = df_clean.iloc[idx[:n_train]]
df_test = df_clean.iloc[idx[n_train:n_train+n_val]]
df_val = df_clean.iloc[idx[n_train+n_val:]]


In [22]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [23]:
y_train = np.log1p(df_train.price.values)
y_test = np.log1p(df_test.price.values)
y_val = np.log1p(df_val.price.values)

In [24]:
del df_train['price']
del df_val['price']
del df_test['price']

### Question 3

* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using `round(score, 2)`
* Which option gives better RMSE?

(Both options are equal 0.64)

In [25]:
X_train_z = df_train.fillna(0)
X_test_z = df_test.fillna(0)
X_val_z = df_val.fillna(0)

X_train_m = df_train.fillna(df_train.reviews_per_month.mean())
X_test_m = df_test.fillna(df_test.reviews_per_month.mean())
X_val_m = df_val.fillna(df_val.reviews_per_month.mean())

* Train Linear Regression

In [26]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [33]:
w0, w = train_linear_regression(X_train_z, y_train)
y_pred_z = w0 + X_train_z.dot(w)

In [34]:
w0, w = train_linear_regression(X_train_m, y_train)
y_pred_m = w0 + X_train_m.dot(w)

In [29]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [38]:
m = rmse(y_pred_m, y_train)
z = rmse(y_pred_z, y_train)
m.round(2), z.round(2)

(0.64, 0.64)

### Question 4

* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0. 
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

If there are multiple options, select the smallest `r`.

(Solution: 0)

In [39]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [43]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w0, w = train_linear_regression_reg(X_train_z, y_train, r=r)

    X_val = df_val
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    print(r, w0, score.round(2))

0 -419.91265863459563 0.63
1e-06 -419.86271586621507 0.63
0.0001 -414.97649263761593 0.63
0.001 -375.27365275360376 0.63
0.01 -191.78384053419808 0.64
0.1 -32.56256054996282 0.66
1 -3.4992168360568994 0.66
5 -0.7033623167239721 0.66
10 -0.3512767588949242 0.66


### Question 5 

* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`.
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores. 
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)


> Note: Standard deviation shows how different the values are.
> If it's low, then all values are approximately the same.
> If it's high, the values are different. 
> If standard deviation of scores is low, then our model is *stable*.